In [ ]:
import h2o
import pandas as pd
import numpy as np
from h2o.grid.grid_search import H2OGridSearch
%matplotlib inline

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321.....

In [ ]:
import h2o 
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
h2o_df = h2o.import_file("http://h2o-public-test-data.s3.amazonaws.com/smalldata/glm_test/Motor_insurance_sweden.txt", sep = '\t')

In [ ]:
df = pd.read_csv("http://h2o-public-test-data.s3.amazonaws.com/smalldata/glm_test/Motor_insurance_sweden.txt", sep = '\t')

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Claims'].hist(bins=100 )

In [ ]:
df['Insured'].hist(bins=100 )

In [ ]:
h2o_df.head()

In [ ]:
#Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
train,test,valid = h2o_df.split_frame(ratios=[.7, .15])

In [ ]:
poisson_fit = H2OGeneralizedLinearEstimator(family = "Gamma")

In [ ]:
poisson_fit.train(y="Claims", x = ["Payment", "Insured", "Kilometres", "Zone", "Bonus", "Make"], training_frame = train)

In [ ]:
predict = poisson_fit.predict(test)

In [ ]:
type(predict)

In [ ]:
predict.shape

In [ ]:
predict

In [ ]:
h2o.export_file(test, 'test_h2o.csv')

In [ ]:
h2o.export_file(predict, 'test_h2o_pred.csv')

In [ ]:
print('Model Type:', poisson_fit.type)
print('R2', poisson_fit.r2(valid = False))

print('RMSLE', poisson_fit.rmsle(valid = False))
print('RMSE', poisson_fit.rmse(valid = False))

print('RMSLE', poisson_fit.rmsle(test))
print('RMSE', poisson_fit.rmse(test))

In [ ]:
# Coefficients that can be applied to the non-standardized data.
print(poisson_fit.coef())

# Coefficients fitted on the standardized data (requires standardize = True, which is on by default)
print(poisson_fit.coef_norm())


In [ ]:
poisson_fit._model_json['output']['coefficients_table']

In [ ]:
poisson_fit._model_json?

In [ ]:
poisson_fit.rmsle?

In [ ]:
sh = poisson_fit.score_history()

In [ ]:
sh

In [ ]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

h2o.init()
# Import a sample binary outcome dataset into H2O
data = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = data.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
data[y] = data[y].asfactor()
test[y] = test[y].asfactor()

# Split data into train & validation
ss = data.split_frame(seed = 1)
train = ss[0]
valid = ss[1]

# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                ntrees=100,
                seed=1)

# Get the grid results, sorted by validation AUC
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='auc', decreasing=True)
gbm_gridperf1

# Grab the top GBM model, chosen by validation AUC
best_gbm1 = gbm_gridperf1.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_gbm_perf1 = best_gbm1.model_performance(test)

best_gbm_perf1.auc()  # 0.7781932261061573


In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
# plot training logloss and auc
sh.plot(x='number_of_trees', y = ['training_auc', 'training_logloss'])

In [ ]:
import os
import pandas as pd

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri    
from rpy2.robjects.packages import importr
